# Demo for `OSGenerator.py`
## Import

In [1]:
from osgenerator import OSGenerator

## Building the graph

In [2]:
osg = OSGenerator()
osg.build("data/friends.txt", "data/checkins.txt")

Building place graph (2/2): 100%|████████████| 6442892/6442892 [01:30<00:00, 71402.15it/s]


## Find all places visited by user `1000`

In [3]:
osg.get_checkins(1000)

[2071063, 3340957, 1946746, 3335022, 3333213]

## When `1000` visited `1946746`
*All `place_id`s are shifted by `1900654`*

In [4]:
osg.get_visittime(1000, 1946746)

[datetime.datetime(2010, 7, 14, 15, 22, 5),
 datetime.datetime(2010, 7, 13, 9, 46, 22)]

## Find user `1000`'s friends

In [5]:
osg.get_friends(1000)

[1, 207, 615, 1395, 4979, 43588]

## Show information about POI `1946746`
This gives information about latitude, longitude, as well as users checked in at this place

In [6]:
osg.get_placeinfo(1946746)

{'placeid': 1946746, 'lat': 51.459313, 'lng': 5.3915300833}

## Find POI `1946746`'svisitors

In [7]:
osg.get_visitor(1946746)

[1000,
 13014,
 16719,
 19889,
 20790,
 27396,
 37388,
 39541,
 45392,
 52605,
 53224,
 63478,
 92068,
 94765,
 97703,
 104088,
 110422,
 112195,
 112813,
 113101,
 120179,
 125152,
 132368,
 132769,
 132780,
 139658,
 142379,
 149669,
 152586,
 153299,
 180505]

## Show OS for user `1000`

In [8]:
os1 = osg.get_object_summary(1000, prettify=True)

{
    "user": 1000,
    "friends": [
        1,
        207,
        615,
        1395,
        4979,
        43588
    ],
    "checksin_at": [
        {
            "placeid": 2071063,
            "time": [
                "16-47-2010 14:07:1279284473"
            ],
            "visitors": [
                1000,
                1715,
                103817
            ],
            "lat": 52.3651886127,
            "lng": 4.9365663942
        },
        {
            "placeid": 3340957,
            "time": [
                "14-18-2010 20:07:1279131512"
            ],
            "visitors": [
                1000
            ],
            "lat": 51.4310359955,
            "lng": 5.4638153315
        },
        {
            "placeid": 1946746,
            "time": [
                "14-22-2010 15:07:1279113725",
                "13-46-2010 09:07:1279007182"
            ],
            "visitors": [
                1000,
                13014,
                16719,
                

## Visualizing user `165697`

In [9]:
import numpy as np
import folium

In [10]:
user = np.random.randint(0, 190000)
user = 165697

os_display = osg.get_object_summary(user, prettify=False)
os_display_coords = [[t['lat'], t['lng']] for t in os_display["checksin_at"]]
os_display_loc = np.mean(os_display_coords, axis=0)

print(f"User: {user}\nNum of checkins: {len(os_display_coords)}")

# with open("data/mapboxkey.txt") as apikey: mapbox_key = next(apikey)
# map_os_display = folium.Map(location=os_display_loc,
#                             tiles=f"https://api.mapbox.com/styles/v1/misaeljordan/clbbbe7ee001214macmpl7fdj/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_key}",
#                             zoom_start=6,
#                             max_zoom=18,
#                             control_scale=True,
#                             attr="© <a href='https://www.mapbox.com/about/maps/'>Mapbox</a>")

stadia_key = "4cb76c74-9556-4900-9d40-4c6ac5a32dc1"
map_os_display = folium.Map(location=os_display_loc,
                            tiles=f"https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{{z}}/{{x}}/{{y}}{{r}}.png?api_key={stadia_key}",
                            zoom_start=6,
                            max_zoom=18,
                            control_scale=True,
                            attr="<a href=https://www.stadiamaps.com/ target=_blank>&copy; Stadia Maps</a>")

[folium.CircleMarker(location=[lat, lng], radius=3, color="#d0d0d0").add_to(map_os_display) for lat, lng in os_display_coords]
map_os_display

User: 165697
Num of checkins: 87


## TODO: define objects' weights and similarity function

### Candidate objects' weight definition
$$o.w_{user = A} = \frac{n\ times\ user\ A\ checked\ in\ at\ o}{total\ check\ ins\ by\ user\ A}$$

### Candidate similarity function definition
1. Jaccard Coefficient
$$ Sim(o_1, o_2) = \frac{|o_1.users\cap o_2.users|}{|o_1.users\cup o_2.users|}$$